In [17]:
import plotly.express as px
import pandas as pd
import json
from pathlib import Path
import numpy as np
from llama.main import compute_metrics_from_files

In [23]:
path_data = Path("../../data/train_class/metrics/epoch_0.json")
with open(path_data) as fp:
    d = json.load(fp)
conf_matrix = np.array(d['conf_matrix'])
accuracy = np.sum(np.diag(conf_matrix)) / np.sum(conf_matrix)

In [24]:
print(accuracy,conf_matrix)

0.258 [[129   0]
 [371   0]]


In [13]:
fig = px.parallel_coordinates(
    df,
    dimensions=["qlora_alpha", "qlora_dropout", "qlora_r", "lr", "epoch", "f1_severe", "f1_non_severe", "accuracy"],
    color="accuracy",
)
fig.show()

In [14]:
fig.write_image(path_data.parent / "paralell.png")

In [15]:
i = 0
conf_matrix = np.array(df["conf_matrix"].iloc[i])
f1 = df['f1'].iloc[i]

In [16]:
compute_metrics_from_files(
    conf_matrix=conf_matrix,
    f1=f1,
    folder_out=path_data.parent
)

c:\Users\robin\Documents\projets\severityPrediction\src\llm\llama\main.py:651: UserWarning:

FigureCanvasAgg is non-interactive, and thus cannot be shown

